In [ ]:
import json
import glob
import os
import tarfile
import re

In [ ]:
PREFIX = "/home/jpivarski/storage/data/physicists/forks-of-cmssw-2022-03-04"

In [ ]:
userrepos = []
for filename in glob.glob(PREFIX + "/user-repos/*.json"):
    userrepos.extend(json.load(open(filename)))

In [ ]:
len(set([repo["full_name"] for repo in userrepos if not repo["fork"]]))

In [ ]:
len([repo for repo in userrepos if not repo["fork"]])

In [ ]:
regex = {
    "": re.compile(r"\b(import\s+uproot[34]*|from\s+uproot[34]*\s+import)\b"),
    "": re.compile(r"\b(import\s+awkward[01]*|from\s+awkward[01]*\s+import)\b"),
    "": re.compile(r"\b(import\s+coffea*|from\s+coffea*\s+import)\b"),
    "": re.compile(r"\b(import\s+ROOT*|from\s+ROOT*\s+import|include\s*[<\"]TFile|include\s*[<\"]TTree|include\s*[<\"]TH[123]|include\s*[<\"]TMVA|include\s*[<\"]ROOT)\b"),
    "": re.compile(r"\b(import\s+numpy*|from\s+numpy*\s+import)\b"),
    "": re.compile(r"\b(import\s+scipy*|from\s+scipy*\s+import)\b"),
    "": re.compile(r"\b(import\s+matplotlib*|from\s+matplotlib*\s+import)\b"),
    "": re.compile(r"\b(import\s+pandas*|from\s+pandas*\s+import)\b"),
    "": re.compile(r"\b(import\s+sklearn*|from\s+sklearn*\s+import)\b"),
    "": re.compile(r"\b(import\s+tensorflow*|from\s+tensorflow*\s+import)\b"),
    "": re.compile(r"\b(import\s+keras*|from\s+keras*\s+import)\b"),
    "": re.compile(r"\b(import\s+torch*|from\s+torch*\s+import)\b"),
    "": re.compile(r"\b(import\s+jax*|from\s+jax*\s+import)\b"),
    "": re.compile(r"\b(import\s+xgboost*|from\s+xgboost*\s+import)\b"),
    "": re.compile(r"\b(import\s+scipy*|from\s+scipy*\s+import)\b"),
}

In [ ]:
python_import = re.compile(r"\bimport\s+([A-Za-z_][A-Za-z_0-9]*)\b")
python_fromimport = re.compile(r"\bfrom\s+([A-Za-z_][A-Za-z_0-9]*)\s+import\b")
cpp_include_bracket = re.compile(r"#include\s*<\s*([^>]+)\s*>")
cpp_include_quote = re.compile(r"\#include\s*\"\s*([^\"]+)\s*\"")

In [ ]:
python_import.search("import  something").group(1)

In [ ]:
python_fromimport.search("from something import *").group(1)

In [ ]:
cpp_include_bracket.search("#include<something>").group(1)

In [ ]:
cpp_include_quote.search("#include\"something\"").group(1)

In [ ]:
for repo in userrepos:
    if not repo["fork"]:
        owner, reponame = repo["full_name"].split("/")
        created_at = datetime.datetime.fromisoformat(repo["created_at"].rstrip("Z"))
        matches = {x: False for x in regex}
        tarname = f"{PREFIX}/actual-repos-2/{owner}-{reponame}.tgz"
        if os.path.exists(tarname):
            with tarfile.open(tarname) as tarball:
                # with tarball.extractfile(f"{owner}-{reponame}/git-log.json") as file:
                #     ages = {}
                #     for commit in git_log["log"]:
                #         timestamp = commit["commit_time"]
                #         for delta in commit["deltas"]:
                #             if delta["new"] not in ages or timestamp > ages[delta["new"]]:
                #                 ages[delta["new"]] = timestamp
                for name in tarball.getnames():
                    if name.endswith(".py"):
                        with tarball.extractfile(name) as file:
                            source = file.read().decode()
                            print(name, created_at)
            break